# Universidad del Valle de Guatemala
## Security Data Science - 10
* Jose Abraham Gutierrez Corado - 19111
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 1: Detección de Pishing

### Importación de Librearias

In [5]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
#from yellowbrick.features import Rank2D
#from yellowbrick.features import RadViz

### Explorar Datos

In [6]:
df = pd.read_csv('dataset_pishing.csv')
df.head(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [7]:
df[df.status == 'legitimate'].sample(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
9713,https://en.wikipedia.org/wiki/Standard_error,0,0,0,0,0,0,0,0,0,...,0,1,0,902,7133,12,0,0,7,legitimate
4338,https://en.wikipedia.org/wiki/United_Nations_S...,0,0,0,0,0,0,0,0,0,...,0,1,0,902,7133,12,0,0,7,legitimate
7392,https://www.mtgowikiprice.com/,0,1,0,0,0,0,0,0,0,...,1,1,0,33,2523,3308718,0,0,2,legitimate
10010,https://remitradar.com/Locator?locationId=4728...,0,0,0,0,0,0,0,0,0,...,1,0,0,85,2107,234160,0,0,5,legitimate
5177,http://www.cannonscombine.com/,0,1,1,0,0,0,0,0,0,...,1,0,0,164,932,0,0,0,1,legitimate


In [8]:
df[df.status == 'phishing'].sample(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
6066,http://gothic-depth-276417.uc.r.appspot.com/ho...,0,0,0,0,0,0,0,0,0,...,1,1,0,228,5616,0,0,1,5,phishing
10189,https://ezoomeranggrow.com/web/english/index.p...,0,0,0,0,0,0,0,0,0,...,1,0,0,305,59,0,0,1,0,phishing
9698,http://u805924gc4.ha004.t.justns.ru/zebzeb/,0,0,0,0,0,0,0,0,0,...,1,0,0,341,2215,627168,0,1,1,phishing
11177,http://email.m-aut.ru/,0,0,0,0,0,0,0,0,0,...,1,0,0,303,61,0,0,1,0,phishing
3328,https://saranewmaroc.com/sur/user/webapps,0,0,0,0,0,0,0,0,0,...,1,1,0,41,325,0,0,1,0,phishing


Si se encuentra balanceado.

In [10]:
df.shape

(11430, 67)

### Derivación de Características
Las características del articulo son:
* f1: Full URL length
* f2: Hostname length
* f4-f20: Special Characters in URL
* f25: HTTPS Token
* f26: Ratio of digits in full URLs
* f27: Ratio of digits in hostnames

In [22]:
# Creamos un nuevo dataframe para que sea más fácil la manipulación de datos
dfNew = df[['url', 'status']]
dfNew.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


### Listado de Partes de una URL
* Protocolo HTTPS
* Subdominio
* Dominio
* TLD
* Ruta
* Parámetro
* Etiqueta

### Funciones

In [25]:
# Obtenemos F1
dfNew['length'] = dfNew['url'].str.len()

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_18656\1539449707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['length'] = dfNew['url'].str.len()


In [46]:
#Obtenemos F2
from urllib.parse import urlparse

def getHostname(dfNew):
    hostnames = []
    for url in dfNew.url:
        hostnames.append('{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(url)))
    
    dfNew['hostname_length'] = hostnames
    
    dfNew['hostname_length'] = dfNew['hostname_length'].str.len()
    
    return dfNew

#Obtenemos la columna de las carateristicas F4 - F20
def SpecialCharacters(dfNew):
    sChars = [".","-", "@", "?", "&", "|", "=", "_", "~", "%", "/", "*", ":", ",", ";", "$", "%20", " "]
    cont = 0
    scCount = []
    for url in dfNew.url:
        for c in url:
            if (c in sChars) and (c != "%"):
                cont += 1
                
            elif c == "%":
                c2 = url.index(c)
                c2 = c + url[c2]
                
                if c2 == "%2":
                    cont += 1
                else:
                    cont += 1
                    
        scCount.append(cont)
        cont = 0
        
    dfNew['Special_Char_Count'] = scCount
    
    return dfNew


In [47]:
dfNew = getHostname(dfNew)
dfNew = SpecialCharacters(dfNew)
dfNew

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_18656\2088770237.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['hostname_length'] = hostnames
C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_18656\2088770237.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['hostname_length'] = dfNew['hostname_length'].str.len()
C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_18656\2088770237.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,url,status,length,hostname_length,Special_Char_Count
0,http://www.crestonwood.com/router.php,legitimate,37,27,7
1,http://shadetreetechnology.com/V4/validation/a...,phishing,77,31,7
2,https://support-appleld.com.secureupdate.duila...,phishing,126,59,19
3,http://rgipt.ac.in,legitimate,18,19,5
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate,55,23,10
...,...,...,...,...,...
11425,http://www.fontspace.com/category/blackletter,legitimate,45,25,7
11426,http://www.budgetbots.com/server.php/Server%20...,phishing,84,26,15
11427,https://www.facebook.com/Interactive-Televisio...,legitimate,105,25,17
11428,http://www.mypublicdomainpictures.com/,legitimate,38,38,6
